In [1]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def transpose(matrix):
    return list(map(list, zip(*matrix)))

def manual_dot(A, B):
    result = [[0 for _ in range(len(B[0]))] for _ in range(len(A))]
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                result[i][j] += A[i][k] * B[k][j]
    return result


In [2]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.W1 = np.random.rand(input_size, hidden_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.rand(hidden_size, output_size)
        self.b2 = np.zeros((1, output_size))

    def forward(self, X):
        A = X.tolist()
        W1 = self.W1.tolist()
        z1 = manual_dot(A, W1)

        for i in range(len(z1)):
            for j in range(len(z1[0])):
                z1[i][j] += self.b1[0][j]
        self.z1 = np.array(z1)
        self.a1 = sigmoid(self.z1)

        A1 = self.a1.tolist()
        W2 = self.W2.tolist()
        z2 = manual_dot(A1, W2)

        for i in range(len(z2)):
            for j in range(len(z2[0])):
                z2[i][j] += self.b2[0][j]
        self.z2 = np.array(z2)
        self.a2 = sigmoid(self.z2)
        return self.a2

    def backward(self, X, y, learning_rate):
        output_error = self.a2 - y
        output_delta = output_error * sigmoid_derivative(self.a2)

        hidden_error = manual_dot(output_delta.tolist(), transpose(self.W2.tolist()))
        hidden_delta = np.array(hidden_error) * sigmoid_derivative(self.a1)

        grad_W2 = manual_dot(transpose(self.a1.tolist()), output_delta.tolist())
        for i in range(len(self.W2)):
            for j in range(len(self.W2[0])):
                self.W2[i][j] -= grad_W2[i][j] * learning_rate

        delta2 = output_delta.tolist()
        for j in range(len(self.b2[0])):
            sum_delta = sum(delta2[i][j] for i in range(len(delta2)))
            self.b2[0][j] -= sum_delta * learning_rate

        grad_W1 = manual_dot(transpose(X.tolist()), hidden_delta.tolist())
        for i in range(len(self.W1)):
            for j in range(len(self.W1[0])):
                self.W1[i][j] -= grad_W1[i][j] * learning_rate

        delta1 = hidden_delta.tolist()
        for j in range(len(self.b1[0])):
            sum_delta = sum(delta1[i][j] for i in range(len(delta1)))
            self.b1[0][j] -= sum_delta * learning_rate

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            self.forward(X)
            self.backward(X, y, learning_rate)

    def predict(self, X):
        return self.forward(X)

In [3]:
X = np.array([
    [0, 0, 0, 0],
    [0, 1, 0, 1],
    [1, 0, 1, 0],
    [1, 1, 1, 1],
    [0, 1, 1, 0]
])

y = np.array([[0], [1], [1], [0], [1]])

nn = NeuralNetwork(input_size=4, hidden_size=3, output_size=1)

nn.train(X, y, epochs=10000, learning_rate=0.1)

output = nn.predict(X)
print("Predictions:")
print(np.round(output, 3))

Predictions:
[[0.044]
 [0.961]
 [0.961]
 [0.043]
 [0.983]]
